In [ ]:
from netCDF4 import Dataset
import numpy as np
import pandas as pd
from matplotlib.cm import get_cmap
import matplotlib.ticker as mticker
import matplotlib.patches as patches

from wrf import getvar, ALL_TIMES, interplevel, get_cartopy, to_np,vinterp

import metpy.calc as mpcalc
from metpy.units import units
import datetime
import matplotlib.pyplot as plt
import numpy as np

import metpy.calc as mpcalc
from metpy.plots import  SkewT
from metpy.units import units

In [ ]:
wrfile = Dataset('/media/lucasdmarten/HD/DADOS_TCC/RESULTADOS/CASO1/MEMBRO1/wrfout_d01_2018-05-22_18:00:00')
names=['pressure','height','temp','dewpoint','humidity_r','moisture','dir','spdknt','THTA','THTE','THTV']
sound = pd.read_csv('/media/lucasdmarten/HD/backup/projetos/webscraping/sounding.c1_23_0Z.csv',skiprows=5,
            delimiter='\s+',names=names)
sound[sound=='|'] = np.nan
sound=sound.interpolate()

In [ ]:
p.sel(Time=slice('2018-05-23','2018-05-25'))

In [ ]:
p = getvar(wrfile, "pressure", timeidx=ALL_TIMES)[:,:,62,57].sel(Time='2018-05-24 00:00:00')* units.hPa
z = getvar(wrfile, "z")[:,62,57]
t = getvar(wrfile, "temp", timeidx=ALL_TIMES)[:,:,62,57].sel(Time='2018-05-24 00:00:00')* units.degC
td = getvar(wrfile, "td", timeidx=ALL_TIMES)[:,:,62,57].sel(Time='2018-05-24 00:00:00')* units.degC
spd_dir = getvar(wrfile, "wspd_wdir", timeidx=ALL_TIMES)
speed = spd_dir[0,:,:62,57].sel(Time='2018-05-24 00:00:00')* units.knots

direction = spd_dir[1,:,:62,57].sel(Time='2018-05-24 00:00:00')* units.degrees
# rh_pblh = interplevel(temp, z, 
#                       sound.height.astype('float').to_list())

In [ ]:
p

In [ ]:
vinter_temp = vinterp(wrfile,temp,'pressure',sound.pressure.astype('float').to_list(),
    extrapolate=False,field_type=None,log_p=True,
               timeidx=0,method='cat',squeeze=True,cache=None,meta=True)[:,62,57]
vinter_height = vinterp(wrfile,z,'pressure',sound.pressure.astype('float').to_list(),
    extrapolate=False,field_type=None,log_p=True,
               timeidx=0,method='cat',squeeze=True,cache=None,meta=True)[:,62,57]
vinter_height

In [ ]:
u

In [ ]:

u, v = mpcalc.wind_components(speed, direction)
plt.rcParams['figure.figsize'] = (9, 9)
fig = plt.figure()

skew = SkewT(fig, rotation=45)

skew.plot(p, t, 'r')
skew.plot(p, td, 'g')
skew.plot_barbs(p, u[:,62], v[:,57])
skew.ax.set_ylim(1000, 100)

skew.plot_dry_adiabats()
skew.plot_moist_adiabats()
skew.plot_mixing_lines()

skew.ax.set_xlim(-50, 60)

fig = plt.gcf()

In [ ]:
names=['pressure','height','temp','dewpoint','humidity_r','moisture','dir','spdknt','THTA','THTE','THTV']
df=pd.read_csv('/media/lucasdmarten/HD/backup/projetos/webscraping/sounding.c1_24_0Z.csv',skiprows=5,delimiter='\s+',names=names)
df[df=='|'] = np.nan
p, td, t = df.pressure.interpolate().astype('float').values * units.hPa, df.dewpoint.interpolate().values * units.degC, df.temp.interpolate().values * units.degC
wind_speed = df['spdknt'].interpolate().astype('float').values * units.knots
wind_dir = df['dir'].interpolate().astype('float').values * units.degrees
u, v = mpcalc.wind_components(wind_speed, wind_dir)
plt.rcParams['figure.figsize'] = (9, 9)
fig = plt.figure()

skew = SkewT(fig, rotation=45)

skew.plot(p, t, 'r')
skew.plot(p, td, 'g')
skew.plot_barbs(p, u, v)
skew.ax.set_ylim(1000, 100)

skew.plot_dry_adiabats()
skew.plot_moist_adiabats()
skew.plot_mixing_lines()

skew.ax.set_xlim(-50, 60)

fig = plt.gcf()